# PSP Data and Switchback Catalogue

## Introduction

This notebook focuses on downloading, processing, and analyzing data from the Parker Solar Probe (PSP) mission. Specifically, we use data from two instruments:

FIELDS (Level 2, magnetic field measurements in RTN coordinates)

SPC (Level 3, solar wind particle measurements including proton and alpha particle fits)

The objective is to combine and process these datasets in order to study the solar wind environment sampled by the PSP during its perihelion passes.

## Importing Required Libraries

We begin by importing the necessary Python libraries:

pandas and numpy for data handling and numerical operations.

pyspedas for direct access to space physics datasets.

astropy and sunpy for handling astronomical times and spacecraft coordinates.

These tools ensure that we can efficiently fetch, transform, and analyze the PSP datasets.

In [21]:
import pandas as pd
import numpy as np 
import pyspedas
from datetime import datetime, timedelta
from sunpy.coordinates import get_horizons_coord
from astropy.time import Time
import astropy.units as u

## Downloading PSP Data

The pyspedas package provides a convenient interface to directly download PSP mission data.

We retrieve Level 2 magnetic field data from the FIELDS instrument (mag_rtn).

We also retrieve Level 3 solar wind data from the SPC instrument (l3i).

The selected time window is November 1st to November 10th, 2018, covering part of PSP’s first perihelion encounter.

By specifying notplot=True and time_clip=True, we ensure that the data is returned in a raw numerical format (not plotting tplot variables) and restricted strictly to the chosen interval.

In [22]:
# Obtained the level 2 data from FIELDS for the time interval between the 1st of November 2018 and the 10th of November 2018
fields = pyspedas.projects.psp.fields(trange=["2018-11-1", "2018-11-10"], datatype='mag_rtn', level='l2', notplot=True, time_clip=True)

# Obtained the level 3 data from SPC for the time interval between the 1st of November 2018 and the 10th of November 2018
spc = pyspedas.projects.psp.spc(trange=["2018-11-1", "2018-11-10"], datatype="l3i", level="l3", notplot=True, time_clip=True)

11-Sep-25 18:26:20: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/fields/l2/mag_rtn/2018/
11-Sep-25 18:26:21: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110100_v02.cdf
11-Sep-25 18:26:23: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110106_v02.cdf
11-Sep-25 18:26:24: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110112_v02.cdf
11-Sep-25 18:26:26: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110118_v02.cdf
11-Sep-25 18:26:27: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110200_v02.cdf
11-Sep-25 18:26:28: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110206_v02.cdf
11-Sep-25 18:26:30: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110212_v02.cdf
11-Sep-25 18:26:31: File is current: psp_data/fields/l2/mag_rtn/2018/psp_fld_l2_mag_rtn_2018110218_v02.cdf
11-Sep-25 18:26:33: File is curren

Using LEVEL=L3


11-Sep-25 18:28:06: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181101_v01.cdf
11-Sep-25 18:28:07: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181102_v01.cdf
11-Sep-25 18:28:09: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181103_v01.cdf
11-Sep-25 18:28:10: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181104_v01.cdf
11-Sep-25 18:28:12: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181105_v01.cdf
11-Sep-25 18:28:13: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181106_v01.cdf
11-Sep-25 18:28:15: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181107_v01.cdf
11-Sep-25 18:28:16: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181108_v01.cdf
11-Sep-25 18:28:18: File is current: psp_data/sweap/spc/l3/l3i/2018/psp_swp_spc_l3i_20181109_v01.cdf
11-Sep-25 18:28:18: Floating point data values for variable na_fit are all fillval (-1.0000

## Constructing DataFrames

Once the datasets are downloaded, we reformat them into pandas DataFrames for easier manipulation:

FIELDS DataFrame: contains timestamp (date) and the three magnetic field components in RTN coordinates (Br, Bt, Bn).

SPC DataFrame: includes timestamp (date), proton density (np_fit), thermal speed (wp_fit), velocity components (vp_fit_R, vp_fit_T, vp_fit_N), as well as analogous parameters for alpha particles.

This structure enables efficient data alignment and further analysis across instruments.

In [23]:
# Create a Dataframe for Parker Solar Probe magnetic field data
# Extract timestamps and magnetic field components in RTN coordinates
fields = pd.DataFrame({
    'date': fields['psp_fld_l2_mag_RTN']['x'],  # Time stamps
    'Br': fields['psp_fld_l2_mag_RTN']['y'][:, 0],  # Radial magnetic field component
    'Bt': fields['psp_fld_l2_mag_RTN']['y'][:, 1],  # Tangential magnetic field component
    'Bn': fields['psp_fld_l2_mag_RTN']['y'][:, 2],  # Normal magnetic field component
})
fields

,date,Br,Bt,Bn
0,2018-11-01 00:00:00.011181312,-35.950676,21.790331,-3.429240
1,2018-11-01 00:00:00.024834560,-36.166656,21.811783,-3.515354
2,2018-11-01 00:00:00.038487808,-36.291637,21.785122,-3.669410
3,2018-11-01 00:00:00.052141056,-36.228504,21.445963,-3.771155
4,2018-11-01 00:00:00.065794560,-36.219070,21.407433,-3.733000
...,...,...,...,...
151617051,2018-11-09 23:59:59.933600768,-52.488533,8.962145,17.353773
151617052,2018-11-09 23:59:59.947254144,-52.260254,8.889447,17.544405
151617053,2018-11-09 23:59:59.960907392,-52.233505,8.884529,17.884466
151617054,2018-11-09 23:59:59.974560896,-52.417992,9.324836,17.994028


In [24]:
# Create a DataFrame for solar wind plasma data from Parker Solar Probe
# Extract proton and alpha particle measurements
spc = pd.DataFrame({
    "date": spc["psp_spc_np_fit"]["x"],  # Time stamps
    "np_fit": spc["psp_spc_np_fit"]["y"],  # Proton number density fit
    "wp_fit": spc["psp_spc_wp_fit"]["y"],  # Proton temperature fit
    "vp_fit_R": spc["psp_spc_vp_fit_RTN"]["y"][:,0],  # Proton velocity radial component
    "vp_fit_T": spc["psp_spc_vp_fit_RTN"]["y"][:,1],  # Proton velocity tangential component
    "vp_fit_N": spc["psp_spc_vp_fit_RTN"]["y"][:,2],  # Proton velocity normal component
    "na_fit": spc["psp_spc_na_fit"]["y"],  # Alpha particle number density fit
    "wa_fit": spc["psp_spc_wa_fit"]["y"],  # Alpha particle temperature fit
    "va_fit_R": spc["psp_spc_va_fit_RTN"]["y"][:,0],  # Alpha velocity radial component
    "va_fit_T": spc["psp_spc_va_fit_RTN"]["y"][:,1],  # Alpha velocity tangential component
    "va_fit_N": spc["psp_spc_va_fit_RTN"]["y"][:,2]   # Alpha velocity normal component
})
spc

,date,np_fit,wp_fit,vp_fit_R,vp_fit_T,vp_fit_N,na_fit,wa_fit,va_fit_R,va_fit_T,va_fit_N
0,2018-10-31 23:59:56.762162944,3.839219,16.745010,484.960785,4.824336,-15.167017,NaN,NaN,NaN,NaN,NaN
1,2018-10-31 23:59:57.144462976,131.806580,90.783508,351.801971,-18.758091,-3.897125,NaN,NaN,NaN,NaN,NaN
2,2018-10-31 23:59:58.018282880,135.694214,87.325546,341.515137,-22.817350,-1.038821,NaN,NaN,NaN,NaN,NaN
3,2018-10-31 23:59:58.892082944,129.804520,86.940903,350.453094,-26.082819,-4.492116,NaN,NaN,NaN,NaN,NaN
4,2018-10-31 23:59:59.765902976,128.642715,82.994934,348.338135,-34.154327,-6.710773,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1468990,2018-11-09 23:59:52.329048192,59.733974,83.093513,463.522797,18.295496,61.938034,NaN,NaN,NaN,NaN,NaN
1468991,2018-11-09 23:59:53.202848256,57.897774,74.417915,458.209290,7.190844,44.558285,NaN,NaN,NaN,NaN,NaN
1468992,2018-11-09 23:59:54.076668288,57.769207,69.906181,456.010193,-10.012024,39.303196,NaN,NaN,NaN,NaN,NaN
1468993,2018-11-09 23:59:54.950488192,53.378601,67.840027,457.422302,9.068673,24.520981,NaN,NaN,NaN,NaN,NaN


## Adding Spacecraft Distance

The solar wind environment depends strongly on heliocentric distance. To account for this:

We generate a 5-minute resolution time array spanning the study interval.

Using sunpy’s get_horizons_coord, we retrieve the spacecraft’s position.

The heliocentric distance is extracted in astronomical units (AU).


In [25]:
# Create time array: Nov 1-10, 2018 (1-minute intervals)
start_time = datetime(2018, 11, 1, 0, 0, 0)
end_time = datetime(2018, 11, 10, 23, 59, 59)

times = []
current = start_time
while current <= end_time:
    times.append(current)
    current += timedelta(minutes=2)


# Get PSP positions
astropy_times = Time(times)
psp_coords = get_horizons_coord('Parker Solar Probe', astropy_times)

# Extract distance directly from the spherical coordinates
distances_au = psp_coords.spherical.distance.to(u.AU).value 

# Create DataFrame
distances_df = pd.DataFrame({
    'date': times,
    'distance_au': distances_au
})

11-Sep-25 18:28:26: Obtained JPL HORIZONS location for Parker Solar Probe (spacecraft) (-96)


INFO: Obtained JPL HORIZONS location for Parker Solar Probe (spacecraft) (-96) [sunpy.coordinates.ephemeris]


## Checking Data Granularity

The cadence (time resolution) of the datasets is not uniform. This is expected because:

Each instrument operates at different sampling rates.

The cadence can also vary depending on PSP’s distance from the Sun.

We compute the time differences between consecutive entries to identify the minimum and maximum sampling intervals. This information is crucial for later steps such as resampling and synchronizing both datasets.

In [26]:
# Calculate time differences between consecutive data points to determine sampling rate
original_granularity_fields = []
for i in range(0, len(fields)-1):
    y = fields["date"][i+1] - fields["date"][i]
    original_granularity_fields.append(y)

# Convert list to pandas Series for easier manipulation
original_granularity_fields = pd.Series(original_granularity_fields)

# Find the most common time intervals (granularities) in the data keeping only intervals that appear more than 10000 times (filtering out outliers)
duplicate_values = original_granularity_fields.value_counts()[original_granularity_fields.value_counts() > 10000].index

# Filter the granularity data to only include the most frequent time intervals
original_granularity_fields = original_granularity_fields[original_granularity_fields.isin(duplicate_values)]

# Display the range of sampling intervals in the magnetic field data
print(f"The minimum granularity is: {min(original_granularity_fields)}")
print(f"The maximum granularity is: {max(original_granularity_fields)}")

The minimum granularity is: 0 days 00:00:00.003345280
The maximum granularity is: 0 days 00:00:00.013653632


In [27]:
# Calculate time differences between consecutive data points for plasma data
original_granularity_spc = []
for i in range(0, len(spc)-1):
    y = spc["date"][i+1] - spc["date"][i]
    original_granularity_spc.append(y)

# Convert list to pandas Series for easier manipulation
original_granularity_spc = pd.Series(original_granularity_spc)

# Find the most common time intervals (granularities) in the SPC plasma data keeping only intervals that appear more than 1000 (filtering out outliers)
duplicate_values = original_granularity_spc.value_counts()[original_granularity_spc.value_counts() > 1000].index

# Filter the granularity data to only include the most frequent time intervals
original_granularity_spc = original_granularity_spc[original_granularity_spc.isin(duplicate_values)]

# Display the range of sampling intervals in the plasma data
print(f"The minimum granularity is: {min(original_granularity_spc)}")
print(f"The maximum granularity is: {max(original_granularity_spc)}")

The minimum granularity is: 0 days 00:00:00.051199872
The maximum granularity is: 0 days 00:00:00.873840128


## Resampling to a Common Cadence

To compare and merge datasets, both must share the same time grid. We resample FIELDS and SPC to a uniform 5-second cadence.

Timestamps are floored to the nearest 5 seconds.

Values within each interval are averaged.

This ensures both datasets are directly comparable and aligned.

In [28]:
# Resample magnetic field data to 5-second intervals
fields["date"] = fields["date"].dt.floor("5s")  # Round timestamps down to nearest 5-second boundary
fields = fields.groupby("date").mean()  # Group by the rounded timestamps and average all measurements within each 5s bin
fields.reset_index(inplace=True)  # Reset index to make 'date' a regular column again

# Resample plasma data to 5-second intervals using the same method
spc["date"] = spc["date"].dt.floor("5s")  # Round timestamps down to nearest 5-second boundary
spc = spc.groupby(["date"]).mean()  # Group by the rounded timestamps and average all measurements within each 5s bin
spc.reset_index(inplace=True)  # Reset index to make 'date' a regular column again

## Merging the DataFrames

After resampling, the FIELDS, SPC and distance DataFrames are merged on their common timestamp column (date).

The resulting solar_data_df contains both magnetic field and plasma parameters, aligned in time for joint analysi

In [29]:
# Merge magnetic field data with plasma data based on timestamps
solar_data_df = pd.merge(left=fields, right=spc, how="left", right_on="date", left_on="date")

# Add distance data to the combined dataset
solar_data_df = pd.merge(left=solar_data_df, right=distances_df, how="left", left_on="date", right_on="date")

# Clean up the DataFrame index after merging
solar_data_df.reset_index(inplace=True)  # Create a new sequential index
solar_data_df.drop("index", axis=1, inplace=True)  # Remove the old index column

solar_data_df

,date,Br,Bt,Bn,np_fit,wp_fit,vp_fit_R,vp_fit_T,vp_fit_N,na_fit,wa_fit,va_fit_R,va_fit_T,va_fit_N,distance_au
0,2018-11-01 00:00:00,-33.844070,24.559757,-0.491868,124.149147,85.248459,357.761963,-22.141657,1.113809,NaN,NaN,NaN,NaN,NaN,0.2389
1,2018-11-01 00:00:05,-34.260448,24.127703,-3.308359,128.375229,86.012474,351.194489,-26.101021,-7.876099,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-11-01 00:00:10,-33.469238,25.651606,-4.649886,133.628006,90.273354,353.808594,-24.602808,-13.071443,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-11-01 00:00:15,-35.103474,22.752834,-6.129280,132.060318,86.126221,348.728546,-30.869125,-17.230963,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-11-01 00:00:20,-36.270771,20.326946,-8.134771,133.955460,85.230675,345.143219,-33.911213,-23.894878,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155515,2018-11-09 23:59:35,-51.887962,12.116952,12.527465,59.910072,84.289673,464.179749,37.831596,41.564037,NaN,NaN,NaN,NaN,NaN,NaN
155516,2018-11-09 23:59:40,-52.460388,4.130184,15.793303,58.790440,79.318794,460.981720,16.959772,53.144073,NaN,NaN,NaN,NaN,NaN,NaN
155517,2018-11-09 23:59:45,-53.124962,-4.578252,15.231713,59.154613,74.733055,459.998413,-4.465960,45.284714,NaN,NaN,NaN,NaN,NaN,NaN
155518,2018-11-09 23:59:50,-52.687485,-0.780584,16.793964,64.689842,81.281548,451.019501,8.400615,48.643848,NaN,NaN,NaN,NaN,NaN,NaN


## Constructing the Switchback Catalogue

A switchback is characterized by a strong deflection of the magnetic field relative to its background direction. To identify them:

1. Compute a rolling average of the magnetic field components (Br, Bt, Bn) using a 2160-sample centered window.

2. Compare the instantaneous field vector with the average vector by computing the angle between them.

3. Define a switchback index (z)

4. Classify an interval as a switchback if z > 0.5

The final DataFrame includes both the continuous parameter z and a boolean switchback flag.

In [30]:
# Calculate rolling window averages for magnetic field components. 2160 points = 3 hours at 5-second resolution (2160 * 5s = 10,800s = 3h)
solar_data_df["Br_window_avg"] = solar_data_df['Br'].rolling(2160, center=True).mean()  # 3-hour centered average of radial component
solar_data_df["Bt_window_avg"]= solar_data_df['Bt'].rolling(2160, center=True).mean()   # 3-hour centered average of tangential component
solar_data_df["Bn_window_avg"] = solar_data_df['Bn'].rolling(2160, center=True).mean()  # 3-hour centered average of normal component

# Create 3D arrays for instantaneous and averaged magnetic field vectors
B = np.stack([solar_data_df["Br"], solar_data_df["Bt"], solar_data_df["Bn"]], axis=1)  # Instantaneous B-field vectors [N x 3]
B_avg = np.stack([solar_data_df["Br_window_avg"], solar_data_df["Bt_window_avg"], solar_data_df["Bn_window_avg"]], axis=1)  # Averaged B-field vectors [N x 3]

# Calculate magnitudes of instantaneous and averaged magnetic field vectors
B_mag = np.linalg.norm(B, axis=1)      # |B| for each time point
B_avg_mag = np.linalg.norm(B_avg, axis=1)  # |B_avg| for each time point

# Handle potential division by zero by setting zero magnitudes to NaN
B_mag[B_mag == 0] = np.nan
B_avg_mag[B_avg_mag == 0] = np.nan

# Calculate unit vectors by normalizing the magnetic field vectors
B_unit = B / B_mag[:, np.newaxis]          # B̂ = B/|B| (unit vector of instantaneous field)
B_avg_unit = B_avg / B_avg_mag[:, np.newaxis]  # B̂_avg = B_avg/|B_avg| (unit vector of averaged field)

# Calculate cosine of angle between instantaneous and averaged magnetic field directions
# cos(α) = B̂ · B̂_avg (dot product of unit vectors)
cos_alpha = np.einsum("ij,ij->i", B_unit, B_avg_unit)
cos_alpha = np.clip(cos_alpha, -1, 1)  # Ensure values stay within valid cosine range [-1, 1]

# Calculate switchback parameter z
# z = 0.5 * (1 - cos(α)), where z=0 means parallel fields, z=1 means antiparallel fields
z = 0.5 * (1 - cos_alpha)

# Store switchback analysis results in the main DataFrame
solar_data_df["z"] = z  # Switchback parameter (0 to 1)
solar_data_df["switchback"] = solar_data_df["z"] > 0.5  # Boolean flag: True if z > 0.5 indicates potential switchback

solar_data_df


,date,Br,Bt,Bn,np_fit,wp_fit,vp_fit_R,vp_fit_T,vp_fit_N,na_fit,wa_fit,va_fit_R,va_fit_T,va_fit_N,distance_au,Br_window_avg,Bt_window_avg,Bn_window_avg,z,switchback
0,2018-11-01 00:00:00,-33.844070,24.559757,-0.491868,124.149147,85.248459,357.761963,-22.141657,1.113809,NaN,NaN,NaN,NaN,NaN,0.2389,NaN,NaN,NaN,NaN,False
1,2018-11-01 00:00:05,-34.260448,24.127703,-3.308359,128.375229,86.012474,351.194489,-26.101021,-7.876099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2018-11-01 00:00:10,-33.469238,25.651606,-4.649886,133.628006,90.273354,353.808594,-24.602808,-13.071443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2018-11-01 00:00:15,-35.103474,22.752834,-6.129280,132.060318,86.126221,348.728546,-30.869125,-17.230963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2018-11-01 00:00:20,-36.270771,20.326946,-8.134771,133.955460,85.230675,345.143219,-33.911213,-23.894878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155515,2018-11-09 23:59:35,-51.887962,12.116952,12.527465,59.910072,84.289673,464.179749,37.831596,41.564037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
155516,2018-11-09 23:59:40,-52.460388,4.130184,15.793303,58.790440,79.318794,460.981720,16.959772,53.144073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
155517,2018-11-09 23:59:45,-53.124962,-4.578252,15.231713,59.154613,74.733055,459.998413,-4.465960,45.284714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
155518,2018-11-09 23:59:50,-52.687485,-0.780584,16.793964,64.689842,81.281548,451.019501,8.400615,48.643848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


## Cleaning the DataFrame

Before exporting, we remove intermediate columns used solely for the switchback calculation (e.g., rolling averages).

- The final DataFrame retains only:

- Magnetic field components (Br, Bt, Bn)

- Proton and alpha parameters from SPC

- Heliocentric distance

- Switchback metrics (z, switchback)

This produces a clean, analysis-ready dataset.

In [31]:
# Remove temporary rolling average columns that were only needed for switchback calculation
solar_data_df = solar_data_df.drop(columns=["Br_window_avg", "Bt_window_avg", "Bn_window_avg"])

# Reorder columns
solar_data_df = solar_data_df[["date",           # Timestamp
                               "Br", "Bt", "Bn", # Magnetic field components (RTN coordinates)
                               "np_fit", "wp_fit", # Proton density and temperature
                               "vp_fit_R", "vp_fit_T", "vp_fit_N", # Proton velocity components (RTN)
                               "na_fit", "wa_fit", # Alpha particle density and temperature  
                               "va_fit_R", "va_fit_T", "va_fit_N", # Alpha velocity components (RTN)
                               "distance_au",      # Distance from Sun in astronomical units
                               "z", "switchback"]] # Switchback analysis results

## Saving the Dataset

Finally, the processed dataset is saved as a CSV file for further use. The CSV will later be uploaded to a GitHub repository alongside this script to ensure full reproducibility and accessibility.

In [32]:
# Save the final processed Parker Solar Probe dataset as a CSV file
solar_data_df.to_csv("/Users/joaquinrobles/Desktop/Final_Project/PSP_data.csv", index=False)